In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
train_file=r'E:/Edvancer/Python/2_Linear Regression/QU/loan_data_train.csv'
test_file=r'E:/Edvancer/Python/2_Linear Regression/QU/loan_data_test.csv'

ld_train=pd.read_csv(train_file)
ld_test=pd.read_csv(test_file)               

In [ ]:
ld_test.head()

In [ ]:
ld_train.head()

In [ ]:
# lets combine the data for data prep

ld_test['Interest.Rate']=np.nan
ld_train['data']='train'
ld_test['data']='test'

#When we add intrest rate as NA, it should be in same as order of train data . I.e, interset rate should come as 4th column only
#hence we use ld_test[ld_train.columns] which ensure that test column order are same as train columns

ld_test=ld_test[ld_train.columns]
ld_all=pd.concat([ld_train,ld_test],axis=0)

In [ ]:
ld_all.head()

In [ ]:
ld_all.dtypes

In [ ]:
# ID,Amount.Funded.By.Investors : drop 
# Interest Rate , Debt to income ratio : remove % and then to numeric
# Amount.Requested , 'Open.CREDIT.Lines','Revolving.CREDIT.Balance': convert it to numeric 
# FICO.Range : replace it by a numeric column which is average of the range
# Employment Length : convert to number
# Loan Lenth, Loan Purpose , State , Home ownership: dummies for categories with good occurence rate

In [ ]:
#axis=1 : Means columns(look for labels and column axis and it will drop that)
#inplace=true (make inplace changes(update there only), Dont want to assign data back to data itself )

ld_all.drop(['ID','Amount.Funded.By.Investors'],axis=1,inplace=True)

In [ ]:
# % will get removed,but it will not convert into char till yet

for col in ['Interest.Rate','Debt.To.Income.Ratio']:
    ld_all[col]=ld_all[col].str.replace("%","")

We can see that many columns which should have really been numbers have been imported as character columns , probably because some characters values in those columns in the files such as % or dot etc . We'll convert all such columns to numbers .

In [ ]:
#pd.to_numeric :- Convert into numeric
#errors="coerce":- whenever it is not poss to  convert into numeric,we/it will set NA 

for col in ['Amount.Requested', 'Interest.Rate','Debt.To.Income.Ratio',
            'Open.CREDIT.Lines','Revolving.CREDIT.Balance']:
    ld_all[col]=pd.to_numeric(ld_all[col],errors='coerce')
    

In [ ]:
ld_all['FICO.Range'].value_counts()

In [ ]:
#astype(float): average might be decimal thats why
#first we split
#then took avg
#k[0]+k[1]/2 :- devide by 2 means 1/2 i.e 0.5

k=ld_all['FICO.Range'].str.split("-",expand=True).astype(float)

ld_all['fico']=0.5*(k[0]+k[1])

del ld_all['FICO.Range']

In [ ]:
ld_all['fico'].value_counts()

In [ ]:
ld_all['Employment.Length'].value_counts()

In [ ]:
#str[:2] means first two

ld_all['Employment.Length']=ld_all['Employment.Length'].str.replace('years',"")

ld_all['Employment.Length']=ld_all['Employment.Length'].str.replace('year',"")

ld_all['Employment.Length']=np.where(ld_all['Employment.Length'].str[:2]=="10",10,ld_all['Employment.Length'])

ld_all['Employment.Length']=np.where(ld_all['Employment.Length'].str[0]=="<",0,ld_all['Employment.Length'])

ld_all['Employment.Length']=pd.to_numeric(ld_all['Employment.Length'],errors='coerce')

In [ ]:
# Notice that to apply string function on pandas data frame columns you need to str attribute

cat_cols=ld_all.select_dtypes(['object']).columns

In [ ]:
cat_cols

In [ ]:
#As we dont want last one i.e data 

cat_cols=cat_cols[:-1]

In [ ]:
cat_cols

You can use below code/loop if you want to ignor categories with low frequencies. In next section logistic regression, we will use panda's get dummies function You can work either of this Ignoring categories with low fre however result in fewer column without affecting model performance too much

In [ ]:
#col in cat_cols:- chheck rowise
#freqs=ld[col].value_counts()

#K=freqs.index[freqs>20][:-1], here [-1] is except the last.That is it will ignor false one,
#and from True one it will create dumnmies except the last thet is n-1 dummies

# name=col+'_'+cat:- Appending cat at end of every dummy var name

#ld[col]==cat :- It give you logical array as true and false
#astype(int):- It will convert true and false to 1 and 0 which gives us a flag variables


for col in cat_cols:
    freqs=ld_all[col].value_counts()
    k=freqs.index[freqs>20][:-1]    #[-1] means except the last i.e it will ignor false one & from True one create n-1 dummies
    for cat in k:
        name=col+'_'+cat
        ld_all[name]=(ld_all[col]==cat).astype(int)
    del ld_all[col]
    print(col)
    
    

In [ ]:
ld_all.shape

In [ ]:
ld_all.isnull().sum()

In [ ]:
#ld_all.isnull().sum():- It will not check in intreset rate and data of training data cz there are no missing value in intrset rate
#intrest rate= 300 is for test data 
#whenever there are missing value in col, we are replace it with the mean of same col belonging to training data 
#why not test?? cz test are we consider as unseen data

for col in ld_all.columns:
    if (col not in ['Interest.Rate','data'])& (ld_all[col].isnull().sum()>0):
        ld_all.loc[ld_all[col].isnull(),col]=ld_all.loc[ld_all['data']=='train',col].mean()

In [ ]:
ld_all.isnull().sum()

In [ ]:
#Training data is where the data indicator column is train
#Once we are seperated, we dont need the data column any more

#From test we always drop response variable and data

ld_train=ld_all[ld_all['data']=='train']
del ld_train['data']
ld_test=ld_all[ld_all['data']=='test']
ld_test.drop(['Interest.Rate','data'],axis=1,inplace=True)

In [ ]:
#Remove overall combine data
del ld_all

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
ld_train1,ld_train2=train_test_split(ld_train,test_size=0.2,random_state=2)

In [ ]:
ld_train.shape

In [ ]:
ld_train1.shape

In [ ]:
ld_train2.shape

In [ ]:
#For any modelling in python, you need to pass predictor and response seperatly
#all predictor will goes in xtrain and response will goes in ytrain

#1 is axis = 1
#we are not using inplace=True cz we are not removing that column from data. 
#Here it will keep all other variables except the Intrest Rate guy


#Build model on ld_train1 using x_train1 and y_train1
x_train1=ld_train1.drop('Interest.Rate',axis=1) 
y_train1=ld_train1['Interest.Rate']             

#check performance on ld_train2 using x_train2 and y_train2
x_train2=ld_train2.drop('Interest.Rate',axis=1) #Predicted/Observed value Value
y_train2=ld_train2['Interest.Rate']  #Actual value

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm=LinearRegression()

In [ ]:
lm.fit(x_train1,y_train1)

In [ ]:
x_train1.shape

In [ ]:
lm.intercept_

In [ ]:
lm.coef_

In [ ]:
#Coefficient corresponding to each of the values/variables

list(zip(x_train1.columns,lm.coef_))

In [ ]:
#Performance Check

predicted_ir=lm.predict(x_train2)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mean_absolute_error(ld_train2['Interest.Rate'],predicted_ir)

In [ ]:
#RMSE

residual=predicted_ir-y_train2
rmse_lm=np.sqrt(np.dot(residual,residual)/len(predicted_ir))
rmse_lm

We know the tentative performance now, lets build the model on entire training to make prediction on test/production

In [ ]:
#For any modelling in python, you need to pass predictor and response seperatly
#all predictor will goes in x_train and response will goes in y_train

#ld_train and ld_test are main train test

#Buid model using ld_train and check performance on ld_test

x_train=ld_train.drop('Interest.Rate',axis=1) #Predictor
y_train=ld_train['Interest.Rate'] #Actual/Response

In [ ]:
lm.fit(x_train,y_train)

In [ ]:
test_pred=lm.predict(ld_test)

We can write these to a csv file for submission like this :

In [ ]:
pd.DataFrame(test_pred).to_csv("mysubmission.csv",index=False)

# Ridge  Regression

We can see that linear regression has produced coefficients for all variables. If you recall our theoretical discussion, we need to penalise coefficient for the variables which are not really contributing well to our resposne and might be causing overfitting of the model. Among the regularised technique we'll first look at Ridge regression.

Since penalty in ridge regression is a hyperparameter , we'd look at multiple values of it and choose the best one through 10 fold cross validation.

In [ ]:
from sklearn.linear_model import Ridge,Lasso
from sklearn.model_selection import GridSearchCV

In [ ]:
#Here lambda is the penalty coefficient and it’s free to take any allowed number while alpha is selected based on the model you want to try
#Try lambda value 1 to 100
#you can take any

lambdas=np.linspace(1,100,100)

In [ ]:
params={'alpha':lambdas}

In [ ]:
#When fit_intercept=True, the line of best fit is allowed to "fit" the y-axis (close to 100 in this example). 
#When fit_intercept=False, the intercept is forced to the origin (0, 0).

model=Ridge(fit_intercept=True)

Gridsearchcv fits the model with best paramter choice on entire train data alredy.

Grid search automatically fits the best estimator on the entire data, you can directly use this to make predictions on test_data. But if you want to look at coefficients , its much more convenient to fit the model with direct function

GridSearchCV implements a “fit” and a “score” method. It also implements “predict”, “predict_proba”, “decision_function”, “transform” and “inverse_transform” if they are implemented in the estimator used.

if you want to look at coefficient seperatly run ridge model seperatly. check looking at coefficients part

In [ ]:
#by default GridSearchCV consider higher the better
#LM consider lower the better, hence if you replace with negative then higher the better becomes a case

grid_search=GridSearchCV(model,param_grid=params,cv=10,scoring='neg_mean_absolute_error')

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
grid_search.best_estimator_

Best value of alpha comes 33.0. So we will try between that ranage only
No need to expand parameter to try out
Try to expand when value comes at edage like near 1 or 100

In [ ]:
#output of grid seartch

grid_search.cv_results_

 if you want you can now fit a ridge regression model with obtained value of alpha , although there is no need, grid search automatically fits the best estimator on the entire data, you can directly use this to make predictions on test_data. But if you want to look at coefficients , its much more convenient to fit the model with direct function

Using the report function given below you can see the cv performance of top few models as well, that will the tentative performance

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
report(grid_search.cv_results_,6)

In [ ]:
test_pred=grid_search.predict(ld_test)

In [ ]:
pd.DataFrame(test_pred).to_csv("mysubmission.csv",index=False)

## For looking at coefficients

In [ ]:
ridge_model=grid_search.best_estimator_

In [ ]:
ridge_model.fit(x_train,y_train)

In [ ]:
list(zip(x_train1.columns,ridge_model.coef_))

You can see that ridge regression though, shrinks the coefficients but never makes them exactly zero, essentially never reduce our model size. Next we look at lasso Regression.

## Lasso Regression

In [ ]:
lambdas=np.linspace(1,10,100)
model=Lasso(fit_intercept=True)
params={'alpha':lambdas}

In [ ]:
grid_search=GridSearchCV(model,param_grid=params,cv=10,scoring='neg_mean_absolute_error')

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
grid_search.best_estimator_

you can see that, the best value of alpha comes at the edge of the range that we tried , we should expand the trial range on that side and run this again

In [ ]:
lambdas=np.linspace(.001,2,100)
params={'alpha':lambdas}

In [ ]:
grid_search=GridSearchCV(model,param_grid=params,cv=10,scoring='neg_mean_absolute_error')
grid_search.fit(x_train,y_train)

In [ ]:
grid_search.best_estimator_

In [ ]:
report(grid_search.cv_results_,5)

In [ ]:
lasso_model=grid_search.best_estimator_

In [ ]:
lasso_model.fit(x_train,y_train)

In [ ]:
list(zip(x_train.columns,lasso_model.coef_))

We can see that lasso regression, not only improves performance on the data slightly , but also makes size of the model smaller by making many coefficents exactly zero, thus excluding them from our model.